In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai import Crew, Agent, Task

international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    # allow_delegation=False,
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    # allow_delegation=False,
)

In [3]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

In [4]:
crew = Crew(
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

 [DEBUG]: == Working Agent: International Chef
 [INFO]: == Starting Task: Come up with a Greek dinner that serves 5 people.


> Entering new CrewAgentExecutor chain...
I need to create a Greek dinner recipe that serves 5 people. This will involve selecting traditional Greek dishes that are easy to prepare at home. 

Action: Ask question to co-worker  
Action Input: {"question": "What traditional Greek dishes would be suitable for a dinner that serves 5 people? I am looking for a complete meal that includes an appetizer, a main course, and a dessert. The dishes should be easy to prepare for home cooks, with a focus on common ingredients and straightforward cooking methods.", "context": "I want to create a Greek dinner menu that is authentic yet accessible for home cooking."}   

I encountered an error while trying to use the tool. This was the error: 'NoneType' object has no attribute 'startswith'.
 Tool Ask question to co-worker accepts these inputs: Ask question to co-worker(question: